In [38]:
from mlscorecheck.auc import (prepare_intervals_for_auc_estimation,
                              auc_from_sens_spec,
                              acc_from_auc,
                              auc_from_sens_spec_kfold,
                              generate_kfold_sens_spec_acc_problem,
                              generate_sens_spec_acc_problem,
                              generate_average,
                              generate_kfold_acc_fix_problem,
                              generate_kfold_auc_fix_problem,
                              generate_kfold_sens_spec_fix_problem,
                              R)
import numpy as np

In [39]:
# min
aucs = []
senss = []
specs = []
for _ in range(10000):
    problem = generate_kfold_sens_spec_fix_problem(0.7, 0.8, 5)
    aucs.append(np.mean(problem['spec'] * problem['sens']))
    senss.append(problem['sens'])
    specs.append(problem['spec'])
np.min(aucs)

np.float64(0.5096269535220292)

In [40]:
auc_from_sens_spec_kfold(scores={'sens': 0.7, 'spec': 0.8},
                         eps=1e-4,
                         p=100,
                         n=100,
                         k=5,
                         lower='min')

(0.5000000000000001, 1.0)

In [4]:
# cmin
aucs = []
senss = []
specs = []
for _ in range(10000):
    problem = generate_kfold_sens_spec_fix_problem(0.7, 0.8, 5)
    if np.any(problem['sens'] < 1 - problem['spec']):
        continue
    aucs.append(np.mean(0.5 + (1 - problem['spec'] - problem['sens'])**2/2.0))
    senss.append(problem['sens'])
    specs.append(problem['spec'])
np.min(aucs)

np.float64(0.6256265491978134)

In [5]:
auc_from_sens_spec_kfold(scores={'sens': 0.7, 'spec': 0.8},
                         eps=1e-4,
                         p=100,
                         n=100,
                         k=2,
                         lower='cmin')

(0.62490002, 0.37998002)

In [2]:
# max
aucs = []
senss = []
specs = []
for _ in range(10000):
    problem = generate_kfold_sens_spec_fix_problem(0.7, 0.8, 5)
    aucs.append(1 - np.mean((1 - problem['spec']) * (1 - problem['sens'])))
    senss.append(problem['sens'])
    specs.append(problem['spec'])
np.max(aucs)

np.float64(0.9912735792344698)

In [5]:
auc_from_sens_spec_kfold(scores={'sens': 0.7, 'spec': 0.8},
                         eps=1e-4,
                         p=100,
                         n=100,
                         k=5,
                         upper='max')

(0.5000000000000001, 1.0)

In [35]:
# amax
aucs = []
senss = []
specs = []
accs0 = []
p = 100
n = 100
for _ in range(10000):
    problem = generate_kfold_sens_spec_fix_problem(0.7, 0.8, 5)
    #if np.any((problem['sens'] + problem['spec'])/2 > 0.75):
    #    continue
    accs = (problem['sens'] + problem['spec']) / 2.0
    aucs.append(1 - np.mean(((1 - accs) * (p + n))**2 / (2*p*n)))
    senss.append(problem['sens'])
    specs.append(problem['spec'])
    accs0.append(accs)
np.max(aucs)

np.float64(0.8749004015858751)

In [27]:
np.array(accs0)[np.array(aucs) == np.max(aucs)]

array([[0.76147792, 0.76286652, 0.73713251, 0.74197881, 0.74654424]])

In [33]:
# max2
aucs = []
senss = []
specs = []
for _ in range(100000):
    problem = generate_kfold_sens_spec_fix_problem(0.7, 0.8, 2)
    aucs.append(np.mean((problem['spec']) * (problem['sens'])))
    senss.append(problem['sens'])
    specs.append(problem['spec'])
np.max(aucs)

np.float64(0.6193357107200045)

In [34]:
np.mean(np.array(R(0.7, 2)) * np.array(R(0.8, 2)))

np.float64(0.62)

In [55]:
sens, spec, problem['auc']

(array([0.44457527, 0.98370375]),
 array([1.62330173, 0.89287064]),
 array([0.7216798, 0.8783202]))

In [66]:
# acc-min-max
accs = []
senss = []
specs = []
aucs = []
p = 100
n = 100
auc = 0.85
for _ in range(10000):
    aucs = generate_average(auc, 2)

    tmp = []
    for a in aucs:
        acc_int = acc_from_auc(scores={'auc': a}, eps=1e-4, p=100, n=100)
        tmp.append(np.random.random()*(acc_int[1] - acc_int[0]) + acc_int[0])
    
    accs.append(np.mean(tmp))
np.min(accs), np.max(accs)

(np.float64(0.6139608747654256), np.float64(0.9226848617721893))

In [64]:
acc_from_auc(scores={'auc': 0.85}, eps=1e-4, p=100, n=100)

(0.6125725874437897, 0.9250499999999999)

In [68]:
# max acc-min-max
accs = []
senss = []
specs = []
aucs = []
p = 100
n = 100
auc = 0.85
for _ in range(10000):
    aucs = generate_average(auc, 2)

    tmp = []
    for a in aucs:
        acc_int = acc_from_auc(scores={'auc': a}, eps=1e-4, p=100, n=100)
        tmp.append(acc_int[1])
    
    accs.append(np.mean(tmp))
np.min(accs), np.max(accs)

(np.float64(0.9250333992608215), np.float64(0.92505))

In [70]:
acc_from_auc(upper='cmax', scores={'auc': 0.85}, eps=1e-4, p=100, n=100)

(0.6125725874437897, 1.0477681991499688)

In [20]:
auc_from_sens_spec_kfold(scores={'sens': 0.7, 'spec': 0.8},
                         eps=1e-4,
                         p=100,
                         n=100,
                         k=5,
                         upper='amax')

(0.5000000000000001, 0.5748999000000001)

In [23]:
RL_acc = R(0.75, 5)
p=100
n=100

1 - np.mean([((1 - acc) * (p + n))**2 / (2*n*p) for acc in RL_acc])

np.float64(0.575)

In [24]:
np.mean((1 - np.array(R(0.8, 5))) * (1 - np.array(R(0.7, 5))))

np.float64(0.2)

In [3]:
generate_average(0.8, 5)

array([0.61848724, 0.96174136, 0.7058372 , 0.78329118, 0.93064302])

In [2]:
problem = generate_kfold_spec_sens_acc_problem(max_acc=True)

In [3]:
auc_from_sens_spec_kfold(scores=problem, eps=1e-4, p=problem['p'], n=problem['n'], k=problem['k'])

(0.3412793103448276, 0.9013499999999999)

In [4]:
problem

{'acc': 0.836697247706422,
 'sens': 0.9012499999999999,
 'spec': 0.6586206896551724,
 'auc': 0.7933836206896551,
 'p': 800,
 'n': 290,
 'k': 5,
 'details': [{'acc': 0.7935779816513762,
   'sens': 0.8875,
   'spec': 0.5344827586206896,
   'p': 160,
   'n': 58,
   'auc': 0.7557112068965518,
   'labels': array([1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
          0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
          1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
          1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
          1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
          1, 1, 